# **<font color='#7868e6'>Fake news detection</font>**

In this notebook, we are going to work on a dataset which consists of 3988 news articles. We will do some text processing and will build a deep learning model which will try to predict whether a particular news is fake or not. This is a Natural Language Processing problem, where we deal with text data and convert texts into some vectors it means a word will be defined as a vector. 


In [ ]:
# libraries for data manipulation and exploration
import pandas as pd
import numpy as np

# for text processing
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# reading the dataset
file_path = '../input/fake-news-detection/data.csv'
data = pd.read_csv(file_path)
data.head()

In [ ]:
# checking the dimension of dataset
data.shape

### **<font color='#7868e6'>Data description</font>**
This dataset contains 4009 rows and 4 columns, here we will see the what each column describe.
- `URLs` : URL of the news
- `Headline` : Headline of the news
- `Body` : News description
- `Label` : **0** for fake news and **1** for true news

In [ ]:
# checking if there is any null value
data.info()

There are few null values in `Body` and `Website` columns, so we can remove those rows because we are dropping only 21 rows, and not loosing much information.

### **<font color='#7868e6'>Analysis</font>**

In [ ]:
# dropping rows where there is any null value
data.dropna(inplace=True, axis=0)

# taking only the name of website from the URLs
pattern = 'https?://([\w.]+)/'
data['Website'] = data.URLs.str.extract(pattern)
data.drop('URLs', axis=1, inplace=True)

In [ ]:
# making a bar plot to count the frequency of 
# fake and real news
sns.countplot(x='Label', 
              data=data,
              palette=['#ffaec0', '#b8b5ff'],
              saturation=1)
sns.despine()
plt.xticks([0,1], ['Fake', 'Real'])
plt.title('Fake Vs Real news');

In our dataset, we have more number of fake news as compare to real news. However, there is no huge difference in the ratio so the dataset is somewhat unbalanced and it is fine. 

We can start working on it.

Let's see which websites contain more number of real or fake news.

In [ ]:
fake_news_count = data[data.Label == 0]['Website'].value_counts()
real_news_count = data[data.Label == 1]['Website'].value_counts()

In [ ]:
fdf = pd.DataFrame({
    'Web':fake_news_count.index,
    'Fake':fake_news_count.values
})

rdf = pd.DataFrame({
    'Web':real_news_count.index,
    'Real':real_news_count.values
})

In [ ]:
rf_count = pd.merge(rdf, fdf, on='Web', how='outer').fillna(0)
rf_count['Real'] = rf_count['Real'].astype(int)
rf_count['Fake'] = rf_count['Fake'].astype(int)
rf_count

Now, we observed that websites are only posting either fake news or real news but not both of them means one website is only posting one type of news either fake or real but not both. 

Most of the websites are posting real news, only four websites are there posting fake news.

So, we want to see the following:
- Top 5 websites posting real news
- Top 5 websites posting fake news

In [ ]:
sns.barplot(y=real_news_count[:5].index, x=real_news_count[:5].values, 
            palette=['#7868e6', '#b8b5ff', '#ffaec0', 'grey', '#a7c5eb'])
sns.despine(bottom=True, left=True)
plt.title('Top 5 websites posting Real News');

In [ ]:
sns.barplot(y=fake_news_count.index, x=fake_news_count.values,
            palette=['#7868e6', '#b8b5ff', '#ffaec0', 'grey'])
sns.despine(bottom=True, left=True)
plt.title('Top websites posting Fake News');

### **<font color='#7868e6'>Text cleaning</font>**
Here, we will remove the unwanted words such as puntuation marks and stop words like - the, is, there, his etc. beacause these words are more frequent and don't have much importance. So, by removing these words we will get important words which will help us to find patterns regarding fake and real news.

 **Steps for text cleaning :**
- convert each word to lower case
- remove any words apart from alphabets
- remove stop words


In [ ]:
lm = WordNetLemmatizer()
def clean_text(column):
    corpus = []
    for value in column:
        # convert each word into lower case
        value = value.lower()
        # take only alphabets
        value = re.sub('[^a-z]', ' ', value)
        value = value.split()
        # lemmatizing those words which are not stop words
        value = [lm.lemmatize(word) for word in value\
                 if word not in set(stopwords.words('english'))]
        corpus.append(' '.join(value))
    return corpus    

# taking both headline and body of news into a single column
data['Text'] = data['Headline'] + ' ' + data['Body']
data['Text'] = clean_text(data.Text)

After cleaning we will try to know about the most frequent words come in real and fake news. To get this thing done, we can use a cloud of words which shows words frequency on the basis of their size means words with bigger fontsize appear mostly as compare to other.

We have combined both **Headline** and **Body** column.

In [ ]:
from wordcloud import WordCloud, STOPWORDS
sw = set(STOPWORDS)

fake_news = data[data.Label == 0]
real_news = data[data.Label == 1]

# wordcloud for fake news
fake_wc = WordCloud(width = 1200, height = 600, 
                    background_color ='white', 
                    stopwords = sw, 
                    min_font_size = 10).generate(' '.join(fake_news.Text)) 
  
# plot the WordCloud image                        
plt.figure(figsize = (12, 6), facecolor = None) 
plt.imshow(fake_wc) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.title('Word Cloud for fake news', fontsize=20);

In [ ]:
# wordcloud for real news
real_wc = WordCloud(width = 1200, height = 600, 
                    background_color ='white', 
                    stopwords = sw, 
                    min_font_size = 10).generate(' '.join(real_news.Text)) 
  
# plot the WordCloud image                        
plt.figure(figsize = (12, 6), facecolor = None) 
plt.imshow(real_wc) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.title('Word Cloud for real news', fontsize=20);

### **<font color='#7868e6'>Modeling</font>**
Here, we are going to build a deep learning model.

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# one hot representation of words
oh_repr = [one_hot(words, 5000) for words in data.Text]

In [ ]:
# list to store length of words in each news
len_list = []
for w in data.Text:
    w = w.split()
    len_list.append(len(w))

print('Summary of word length :')
pd.Series(len_list).describe()

In [ ]:
# taking sentences length as 400
sent_length = 400
# padding
embedded_doc = pad_sequences(oh_repr, padding='pre', maxlen=sent_length)
embedded_doc[:5]

In [ ]:
# initializing model
model = Sequential()
# adding embedding layer
model.add(Embedding(5000, 100, input_length=sent_length))
model.add(LSTM(150))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
x = np.array(embedded_doc)
y = data.Label

# splitting the dataset into train and test
train_x, test_x, train_y, test_y = train_test_split(x, y, 
                                                    test_size=0.25, random_state=3)

In [ ]:
# fitting the model
history = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=10, batch_size=64)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
pred = model.predict_classes(test_x);

In [ ]:
# plotting graph for confusion matrix
cm = confusion_matrix(test_y, pred)
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt='', cbar=False, linewidths=2,
            xticklabels = ['Fake','Real'], yticklabels = ['Fake','Real'])
plt.title('confusion matrix')
plt.xlabel('Predicted', color='navy', fontsize=15)
plt.ylabel('Actual', color='navy', fontsize=15);

In [ ]:
# accuracy of the model we built
accuracy_score(test_y, pred)

So, we built a LSTM model with 97% accuracy which is quit amazing. We can try other parameters in order to increase the accuracy.